# Equal-Weight S&P 500 Index Fund

## Introduction & Library Imports

The S&P 500 is the world's most popular stock market index. The largest fund that is benchmarked to this index is the SPDR® S&P 500® ETF Trust. It has more than US$250 billion of assets under management.

The goal of this section of the course is to create a Python script that will accept the value of your portfolio and tell you how many shares of each S&P 500 constituent you should purchase to get an equal-weight version of the index fund.

## Library Imports

The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [2]:
import pandas as pd
import numpy as np
import requests

## Importing Our List of Stocks

The next thing we need to do is import the constituents of the S&P 500.

These constituents change over time, so in an ideal world you would connect directly to the index provider (Standard & Poor's) and pull their real-time constituents on a regular basis.

Paying for access to the index provider's API is outside of the scope of this course. 

There's a static version of the S&P 500 constituents available here. [Click this link to download them now](https://drive.google.com/file/d/1ZJSpbY69DVckVZlO9cC6KkgfSufybcHN/view?usp=sharing). Move this file into the `starter-files` folder so it can be accessed by other files in that directory.

Now it's time to import these stocks to our Jupyter Notebook file.

In [64]:
stocks = pd.read_csv('constituents.csv')
symbols = stocks['Symbol'].sort_values()
symbols

9         A
26      AAL
40     AAPL
3      ABBV
11     ABNB
       ... 
498     YUM
500     ZBH
499    ZBRA
501    ZION
502     ZTS
Name: Symbol, Length: 503, dtype: object

## Acquiring an API Token

Now it's time to import our IEX Cloud API token. This is the data provider that we will be using throughout this course.

API tokens (and other sensitive information) should be stored in a `secrets.py` file that doesn't get pushed to your local Git repository. We'll be using a sandbox API token in this course, which means that the data we'll use is randomly-generated and (more importantly) has no cost associated with it.

[Click here](http://nickmccullum.com/algorithmic-trading-python/secrets.py) to download your `secrets.py` file. Move the file into the same directory as this Jupyter Notebook before proceeding.

In [4]:
from secrets_api import IEX_CLOUD_API_TOKEN

## Making Our First API Call

Now it's time to structure our API calls to IEX cloud. 

We need the following information from the API:

* Market capitalization for each stock
* Price of each stock



In [5]:
symbol = 'AAPL'
url = f'https://cloud.iexapis.com/stable/stock/{symbol}/quote?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(url).json()
print(data['latestPrice'])

197.96


## Parsing Our API Call

The API call that we executed in the last code block contains all of the information required to build our equal-weight S&P 500 strategy. 

With that said, the data isn't in a proper format yet. We need to parse it first.

In [6]:
price =  data['latestPrice']
marketcap = data['marketCap']

## Adding Our Stocks Data to a Pandas DataFrame

The next thing we need to do is add our stock's price and market capitalization to a pandas DataFrame. Think of a DataFrame like the Python version of a spreadsheet. It stores tabular data.

In [7]:
my_columns = ['Ticker', 'Price', 'Market Captalization', 'Number of Shares to Buy']
my_df = pd.DataFrame(columns=my_columns)
my_df

,Ticker,Price,Market Captalization,Number of Shares to Buy


In [13]:
my_series = pd.Series([symbol,price,marketcap,'NaN'],index=my_columns)
my_series
mylist = [symbol,price,marketcap,'NaN'] 
my_df.append(
    my_series,
    ignore_index=True,
)

/var/folders/mj/9ptdvvyj6vd_d80g6y817crw0000gn/T/ipykernel_26959/4281834600.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  my_df.append(


,Ticker,Price,Market Captalization,Number of Shares to Buy
0,AAPL,197.96,3078822785920,NaN


## Looping Through The Tickers in Our List of Stocks

Using the same logic that we outlined above, we can pull data for all S&P 500 stocks and store their data in the DataFrame using a `for` loop.

In [70]:
my_df = pd.DataFrame(columns=my_columns)
for ticker in symbols[:5]:
    url = f'https://cloud.iexapis.com/stable/stock/{ticker}/quote?token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(url).json()
    new_row = pd.DataFrame(
        [ticker, data['latestPrice'], data['marketCap'], 'NaN'],
        index=my_columns
    ).transpose()
    my_df = pd.concat([my_df,new_row],ignore_index=True)
my_df

,Ticker,Price,Market Captalization,Number of Shares to Buy
0,A,133.74,39130621891,NaN
1,AAL,14.09,9208386350,NaN
2,AAPL,197.96,3078822785920,NaN
3,ABBV,154.3,272422424060,NaN
4,ABNB,144.51,92829926715,NaN


## Using Batch API Calls to Improve Performance

Batch API calls are one of the easiest ways to improve the performance of your code.

This is because HTTP requests are typically one of the slowest components of a script.

Also, API providers will often give you discounted rates for using batch API calls since they are easier for the API provider to respond to.

IEX Cloud limits their batch API calls to 100 tickers per request. Still, this reduces the number of API calls we'll make in this section from 500 to 5 - huge improvement! In this section, we'll split our list of stocks into groups of 100 and then make a batch API call for each group.

In [24]:
def chunks(lst,n):
    for i in range(0, len(lst), n):
        yield lst[i:i+n]

In [73]:
chunk_list = chunks(symbols, 100)
my_df = pd.DataFrame(columns=my_columns)
print(chunk_list)
chunk_strings = []
for chunk in chunk_list:
    chunk_string = ','.join(chunk)
    chunk_strings.append(chunk_string)
    batch_api_url = f"https://cloud.iexapis.com/stable/stock/market/batch?symbols={chunk_string}&types=quote&token={IEX_CLOUD_API_TOKEN}"
    #print(batch_api_url)
    data = requests.get(batch_api_url).json()
    for string in chunk_string.split(','):
        new_row = pd.DataFrame(
            [string,data[string]['quote']['latestPrice'], data[string]['quote']['marketCap'],'NaN'],
            index = my_columns
        ).transpose()
        print(new_row)
        my_df = pd.concat([my_df,new_row],ignore_index=True)    

<generator object chunks at 0x12063e200>
  Ticker   Price Market Captalization Number of Shares to Buy
0      A  133.74          39130621891                     NaN
  Ticker  Price Market Captalization Number of Shares to Buy
0    AAL  14.09           9208386350                     NaN
  Ticker   Price Market Captalization Number of Shares to Buy
0   AAPL  197.96        3078822785920                     NaN
  Ticker  Price Market Captalization Number of Shares to Buy
0   ABBV  154.3         272422424060                     NaN
  Ticker   Price Market Captalization Number of Shares to Buy
0   ABNB  144.51          92829926715                     NaN
  Ticker   Price Market Captalization Number of Shares to Buy
0    ABT  107.25         186192277986                     NaN
  Ticker  Price Market Captalization Number of Shares to Buy
0   ACGL  78.38          29249214227                     NaN
  Ticker   Price Market Captalization Number of Shares to Buy
0    ACN  343.64         2284472765

In [75]:
my_df

,Ticker,Price,Market Captalization,Number of Shares to Buy
0,A,133.74,39130621891,NaN
1,AAL,14.09,9208386350,NaN
2,AAPL,197.96,3078822785920,NaN
3,ABBV,154.3,272422424060,NaN
4,ABNB,144.51,92829926715,NaN
...,...,...,...,...
498,YUM,130.93,36700755114,NaN
499,ZBH,117.9,24638825827,NaN
500,ZBRA,244.43,12553828006,NaN
501,ZION,41.82,6195582774,NaN


## Calculating the Number of Shares to Buy

As you can see in the DataFrame above, we stil haven't calculated the number of shares of each stock to buy.

We'll do that next.

In [77]:
try:
    portfolio = float(input("Please input your portfolio"))
except ValueError:
    print("That's not a number")
    portfolio = float(input("Please input your portfolio"))

In [89]:
#my_df['Number of Shares to Buy'] = (portfolio/len(my_df.index))//my_df['Price']
#my_df['Number of Shares to Buy'] = [int((portfolio / len(my_df.index)) // price) for price in my_df['Price']]
my_df['Number of Shares to Buy'] = my_df.apply(lambda x: int((portfolio / len(my_df.index)) // x['Price']), axis=1)
my_df

,Ticker,Price,Market Captalization,Number of Shares to Buy
0,A,133.74,39130621891,14
1,AAL,14.09,9208386350,141
2,AAPL,197.96,3078822785920,10
3,ABBV,154.3,272422424060,12
4,ABNB,144.51,92829926715,13
...,...,...,...,...
498,YUM,130.93,36700755114,15
499,ZBH,117.9,24638825827,16
500,ZBRA,244.43,12553828006,8
501,ZION,41.82,6195582774,47


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

### Initializing our XlsxWriter Object

In [101]:
import pandas as pd

writer = pd.ExcelWriter("recommended_trades.xlsx", engine='xlsxwriter')

my_df.to_excel(writer, sheet_name="Recommended Trades", index=False)

### Creating the Formats We'll Need For Our `.xlsx` File

Formats include colors, fonts, and also symbols like `%` and `$`. We'll need four main formats for our Excel document:
* String format for tickers
* \\$XX.XX format for stock prices
* \\$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase

In [102]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_format = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_format = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_format = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

### Applying the Formats to the Columns of Our `.xlsx` File

We can use the `set_column` method applied to the `writer.sheets['Recommended Trades']` object to apply formats to specific columns of our spreadsheets.

Here's an example:

```python
writer.sheets['Recommended Trades'].set_column('B:B', #This tells the method to apply the format to column B
                     18, #This tells the method to apply a column width of 18 pixels
                     string_template #This applies the format 'string_template' to the column
                    )
```

In [ ]:
# writer.sheets['Recommended Trades'].write('A1', 'Ticker', string_format)
# writer.sheets['Recommended Trades'].write('B1', 'Price', string_format)
# writer.sheets['Recommended Trades'].write('C1', 'Market Capitalization', string_format)
# writer.sheets['Recommended Trades'].write('D1', 'Number Of Shares to Buy', string_format)
# writer.sheets['Recommended Trades'].set_column('A:A', 20, string_format)
# writer.sheets['Recommended Trades'].set_column('B:B', 20, dollar_format)
# writer.sheets['Recommended Trades'].set_column('C:C', 20, dollar_format)
# writer.sheets['Recommended Trades'].set_column('D:D', 20, integer_format)

This code works, but it violates the software principle of "Don't Repeat Yourself". 

Let's simplify this by putting it in 2 loops:

In [103]:
column_formats = {
    'A': ['Ticker', string_format],
    'B': ['Price', dollar_format],
    'C': ['Market Capitalization', dollar_format],
    'D': ['Number of Shares to Buy', integer_format]
}

for key in column_formats.keys():
    writer.sheets['Recommended Trades'].write(f'{key}1', column_formats[key][0], string_format)
    writer.sheets['Recommended Trades'].set_column(f'{key}:{key}', 20, column_formats[key][1])

## Saving Our Excel Output

Saving our Excel file is very easy:

In [104]:
writer.save()

/var/folders/mj/9ptdvvyj6vd_d80g6y817crw0000gn/T/ipykernel_26959/934276808.py:1: FutureWarning: save is not part of the public API, usage can give unexpected results and will be removed in a future version
  writer.save()
